# Task 5: A/B-тестирование

Итак... давайте, наконец, выясним, повлияло ли посещение ленты новостей на поведение учащихся. Они в итоге приступили раньше к работе над лабораторным заданием? Помните, что у нас есть две подготовленные таблицы в базе данных: `test` и `control`. Мы выполним нечто, похожее на A/B-тестирование. Чтобы поймать эффект, нам нужно вычислить значение дельты (период времени между датой первого коммита и сроком сдачи лабораторного задания) до того момента, когда учащиеся впервые посетили страницу с лентой новостей, и после этого. Мы должны сделать то же самое и для контрольной группы.

Другими словами, каждый пользователь из тестовой таблицы имеет свою собственную временную метку для первого посещения новостной ленты. Мы хотим вычислить среднее значение дельты (разницу между датой первого коммита и сроком сдачи) до этой временной метки и после нее. Мы сделаем то же самое для пользователей в контрольной группе. Вы можете сказать: «Но они вообще не посещали ленту новостей». Это так, и ранее мы решили использовать среднюю временную метку первого просмотра пользователями тестовой группы для пользователей контрольной группы.

Если значение дельты перед первым посещением ленты новостей значительно отличается от этого показателя после первого посещения в тестовой группе, и мы не видим аналогичного эффекта в контрольной группе, значит, создание страницы с новостной лентой было отличной идеей. Мы можем распространить эту практику на всю группу.

Подробное описание:

## 1. Создайте соединение с базой данных с помощью библиотеки sqlite3

In [1]:
import pandas as pd
import sqlite3 as s
connection = s.connect('C:/Users/зщ/Downloads/DS_project11-0-develop/src/data/checking-logs.sqlite') 

Используя только один запрос для каждой из групп, создайте два датафрейма:
`test_results` и `control_results` со столбцами `time` и `avg_diff` и только двумя строками.
* `times` должно иметь значения `before` и `after`.
* `avg_diff` содержит среднее значение дельты для всех пользователей за период времени до первого посещения ленты новостей каждым из них и после этого.
* Учитываются только те пользователи, для которых имеются наблюдения и `before`, и `after`.

In [2]:
sql = """WITH a AS
                (
                SELECT "before" as times,AVG((strftime("%s", test.first_commit_ts)-deadlines.deadlines)/3600) as avg_diff
                FROM
                test LEFT JOIN deadlines ON test.labname = deadlines.labs
                WHERE test.labname != "project1" AND test.first_commit_ts < test.first_view_ts
                GROUP BY test.uid
                HAVING test.uid IN(SELECT DISTINCT uid FROM test WHERE labname != "project1" AND first_commit_ts > first_view_ts)

                union

                SELECT "after" as times,avg((strftime("%s", test.first_commit_ts)-deadlines.deadlines)/3600) as avg_diff
                FROM test LEFT JOIN deadlines ON test.labname = deadlines.labs
                WHERE test.labname != "project1" AND test.first_commit_ts > test.first_view_ts
                GROUP BY test.uid
                HAVING test.uid IN(SELECT DISTINCT uid FROM test WHERE labname != "project1" AND first_commit_ts < first_view_ts)
                )

                SELECT a.times, AVG(a.avg_diff) as avg_diff
                FROM a GROUP BY a.times
                ;"""
test_results = pd.read_sql(sql, connection)
test_results

,times,avg_diff
0,after,-99.523810
1,before,-66.047619


In [3]:
sql2 = """WITH a AS
                (
                SELECT"before" AS times,AVG((strftime("%s", control.first_commit_ts)-deadlines.deadlines) / 3600) as avg_diff
                FROM control LEFT JOIN deadlines ON control.labname = deadlines.labs
                WHERE control.labname != "project1" AND control.first_commit_ts < 
                (SELECT avg_time FROM(SELECT DISTINCT uid, datetime(AVG(strftime("%s", first_view_ts)), "unixepoch") as avg_time FROM test))
                GROUP BY control.uid
                HAVING control.uid IN (SELECT DISTINCT uid FROM control WHERE labname != "project1" AND first_commit_ts > (SELECT avg_time FROM 
                (SELECT DISTINCT uid, datetime(AVG(strftime("%s", first_view_ts)), "unixepoch") as avg_time FROM test)))

                union

                SELECT"after" as times,AVG((strftime("%s", control.first_commit_ts)-deadlines.deadlines)/3600) as avg_diff 
                FROM control LEFT JOIN deadlines 
                ON control.labname = deadlines.labs
                WHERE control.labname != "project1" AND control.first_commit_ts > (SELECT avg_time FROM(SELECT DISTINCT uid, datetime(AVG(strftime("%s", first_view_ts)), "unixepoch") as avg_time 
                FROM test))
                GROUP BY control.uid
                HAVING control.uid in(SELECT DISTINCT uid FROM control WHERE labname != "project1" AND first_commit_ts < (SELECT avg_time FROM 
                (SELECT DISTINCT uid, datetime(AVG(strftime("%s", first_view_ts)), "unixepoch") as avg_time FROM test))))
                
                SELECT a.times,avg(a.avg_diff) as avg_diff FROM a GROUP BY a.times
                ;"""
control_results = pd.read_sql(sql2, connection)
control_results

,times,avg_diff
0,after,-99.322222
1,before,-98.033333


In [4]:
connection.close()

## 5. Дайте ответ: оказалось ли предположение верным и влияет ли наличие страницы с новостной лентой на поведение учащихся?

_Изменение времени в тестовой группе:_

In [5]:
test_results #проверка

,times,avg_diff
0,after,-99.523810
1,before,-66.047619


_Изменение времени в контрольной группе:_

In [6]:
control_results #проверка

,times,avg_diff
0,after,-99.322222
1,before,-98.033333


Дайте ответ: оказалось ли предположение верным и влияет ли наличие страницы с новостной лентой на поведение учащихся?
**Да, наличие страницы с новостной лентой влияет на поведение учащихся**